半监督学习（Semi-supervised learning）应用于训练数据部分无标签场景。

利用这些附加的未标记数据来更好地`捕获底层数据分布的形状`，并将其更好地类推广到新的样本。`当训练数据中有非常少量的有标签的点和大量的无标签的点时，这些算法可以表现良好。`

一般将无标签的数据，标记为-1。

## 自训练 -- self-training

自训练的实现: 一个给定的监督分类器可以作为一个半监督分类器发挥作用，使其能够从未标记的数据中学习，`生成伪标签`。

In [15]:
import numpy as np 
from sklearn import datasets
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC

iris = datasets.load_iris()

# 随机造一些无标签数据行
rng = np.random.RandomState(42)
random_unlabeled_points = rng.rand(iris.target.shape[0]) < 0.3
iris.target[random_unlabeled_points] = -1

iris.target

array([ 0,  0,  0,  0, -1, -1, -1,  0,  0,  0, -1,  0,  0, -1, -1, -1,  0,
        0,  0, -1,  0, -1, -1,  0,  0,  0, -1,  0,  0, -1,  0, -1, -1,  0,
        0,  0,  0, -1,  0,  0, -1,  0, -1,  0, -1,  0,  0,  0,  0, -1,  1,
        1,  1,  1,  1,  1, -1, -1, -1,  1,  1, -1,  1,  1, -1,  1, -1,  1,
       -1,  1,  1, -1, -1,  1,  1,  1,  1, -1,  1, -1,  1,  1,  1, -1,  1,
        1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1,  2,
        2,  2,  2, -1,  2,  2, -1, -1, -1, -1,  2,  2,  2,  2,  2, -1,  2,
        2,  2,  2,  2, -1, -1,  2,  2,  2, -1,  2,  2, -1, -1,  2,  2,  2,
        2,  2,  2,  2,  2, -1,  2,  2, -1, -1,  2,  2, -1, -1])

In [17]:
svc = SVC(probability=True, gamma="auto")

# 默认按照阈值>0.75生成伪标签
self_training_model = SelfTrainingClassifier(svc)
self_training_model.fit(iris.data, iris.target)
self_training_model.predict(iris.data)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

参数`criterion`，用于选择将哪些标签添加到训练集的选择标准:
- 如果是 "threshold"，预测概率高于阈值的伪标签被添加到数据集。
- 如果是 "k_best"，具有最高预测概率的k_best伪标签被添加到数据集。

当使用 "threshold"时，应该使用一个经过良好校准的分类器。

In [20]:
from sklearn.metrics import accuracy_score

self_training_model2 = SelfTrainingClassifier(svc,criterion='k_best',k_best=2)
self_training_model2.fit(iris.data, iris.target)

# 准确率不是很高，好像效果不太行~
accuracy_score(iris.target,self_training_model2.predict(iris.data))

0.66

## 标签传播 -- Label Propagation

标签传播指的是`半监督图推理算法`的变种。

原理：`未标注的观察值的结构与类的结构一致`，因此类的标签可以传播到训练集的未标注的观察值。

该模型特点：
- 用于分类任务
- 用核方法将数据投射到交替的维度空间中

In [23]:
import numpy as np
from sklearn import datasets
from sklearn.semi_supervised import LabelPropagation
from sklearn.metrics import accuracy_score

label_prop_model = LabelPropagation()

iris = datasets.load_iris()
rng = np.random.RandomState(42)
random_unlabeled_points = rng.rand(len(iris.target)) < 0.3
labels = np.copy(iris.target)
labels[random_unlabeled_points] = -1

label_prop_model.fit(iris.data, labels)
# 这个方法比自训练效果好很多
accuracy_score(iris.target,label_prop_model.predict(iris.data))

0.9866666666666667